In [198]:
# part 1 and 2
import re
from heapq import heapify, heappush, heappop

# lines = open('inputs/input_16_test.txt','r').read().splitlines()
lines = open('inputs/input_16.txt','r').read().splitlines()

valves = {}
for line in lines:
    doubles = [x for x in re.findall(r'([A-Z]{2})', line)]
    a = re.search(r'\d+', line) # find integer 
    flow = int(a.group(0))
    valves[doubles[0]] = flow,doubles[1:]

_ = [print(x,valves[x]) for x in valves]


SY (23, ['CS'])
VG (15, ['XZ', 'FT', 'PF', 'TF', 'LG'])
LQ (0, ['YI', 'PW'])
ON (0, ['AA', 'KC'])
EX (0, ['GC', 'CB'])
JV (0, ['XI', 'QT'])
UU (0, ['YI', 'CB'])
JS (8, ['WE', 'JN', 'YN', 'KC', 'WC'])
WE (0, ['YI', 'JS'])
YN (0, ['JS', 'PW'])
LG (0, ['BO', 'VG'])
GE (0, ['AA', 'HU'])
AA (0, ['VP', 'ON', 'PC', 'GE', 'YO'])
GC (20, ['DG', 'ZK', 'WC', 'KF', 'EX'])
QO (12, ['YA', 'TF', 'BV'])
JN (0, ['JS', 'ZX'])
WO (0, ['ZW', 'XI'])
YO (0, ['AA', 'AY'])
TF (0, ['VG', 'QO'])
PF (0, ['BY', 'VG'])
DO (0, ['KF', 'AY'])
FG (0, ['AY', 'PW'])
VR (24, ['IC', 'CH'])
CH (0, ['VR', 'NW'])
CB (9, ['QZ', 'BP', 'UU', 'JW', 'EX'])
YA (0, ['QO', 'RT'])
KF (0, ['DO', 'GC'])
DP (0, ['QT', 'ZK'])
YI (3, ['PC', 'CY', 'WE', 'LQ', 'UU'])
HU (0, ['PW', 'GE'])
PC (0, ['AA', 'YI'])
FT (0, ['XI', 'VG'])
AY (19, ['ZX', 'CY', 'FG', 'DO', 'YO'])
RT (21, ['YA', 'BO'])
ZK (0, ['GC', 'DP'])
PW (6, ['YN', 'FG', 'BP', 'LQ', 'HU'])
ZW (14, ['BY', 'WO'])
BP (0, ['CB', 'PW'])
ZX (0, ['JN', 'AY'])
IC (0, ['QZ', 'VR'])
BV (0, [

In [ ]:
# part 1
def move(dP, clock, loc, openset):
    for newloc in valves[loc][1]:
        heappush(todo,(dP, clock, newloc, openset))
                
def openvalve(dP, clock, loc, openset):
    if loc not in openset and valves[loc][0] > 0:
        dP -= valves[loc][0]*(30-clock)
        heappush(todo, (dP, clock, loc, (*openset,loc)))

print(valves)
# completed = [] # heap of completed states
dP_biggest_drop = 0
visited = {}  # dictionary of cache of visited, key: loc,openset, value: clock
todo = [(0,0,'AA',())]  # starting state
clock = 0
cnt = 0
while todo != []:
    cnt += 1
    dP, clock, loc, openset = heappop(todo)
    if cnt % 1e7 == 0:
        print(f'cnt = {cnt}, current pop = {dP, clock, loc, openset}')
    if (loc,openset) in visited and clock > visited[(loc,openset)]:
        continue # move on to next pop
    if dP < dP_biggest_drop:
        dP_biggest_drop = dP
        print(f'Best so far: {dP, clock, loc, openset}')
    visited[(loc,openset)] = clock
    clock += 1
    if clock <= 29:
        if valves[loc][0] > 0:
            openvalve(dP, clock, loc, openset)
        move(dP, clock, loc, openset)

print(f'cnt = {cnt}, current pop = {dP, clock, loc, openset}')
# test case: Best so far: (-1651, 24, 'CC', ('DD', 'BB', 'JJ', 'HH', 'EE', 'CC'))
# 16 sec, cnt = 12171829, Best so far: (-2114, 28, 'PW', ('AY', 'GC', 'WT', 'SY', 'VR', 'CB', 'PW'))

{'SY': (23, ['CS']), 'VG': (15, ['XZ', 'FT', 'PF', 'TF', 'LG']), 'LQ': (0, ['YI', 'PW']), 'ON': (0, ['AA', 'KC']), 'EX': (0, ['GC', 'CB']), 'JV': (0, ['XI', 'QT']), 'UU': (0, ['YI', 'CB']), 'JS': (8, ['WE', 'JN', 'YN', 'KC', 'WC']), 'WE': (0, ['YI', 'JS']), 'YN': (0, ['JS', 'PW']), 'LG': (0, ['BO', 'VG']), 'GE': (0, ['AA', 'HU']), 'AA': (0, ['VP', 'ON', 'PC', 'GE', 'YO']), 'GC': (20, ['DG', 'ZK', 'WC', 'KF', 'EX']), 'QO': (12, ['YA', 'TF', 'BV']), 'JN': (0, ['JS', 'ZX']), 'WO': (0, ['ZW', 'XI']), 'YO': (0, ['AA', 'AY']), 'TF': (0, ['VG', 'QO']), 'PF': (0, ['BY', 'VG']), 'DO': (0, ['KF', 'AY']), 'FG': (0, ['AY', 'PW']), 'VR': (24, ['IC', 'CH']), 'CH': (0, ['VR', 'NW']), 'CB': (9, ['QZ', 'BP', 'UU', 'JW', 'EX']), 'YA': (0, ['QO', 'RT']), 'KF': (0, ['DO', 'GC']), 'DP': (0, ['QT', 'ZK']), 'YI': (3, ['PC', 'CY', 'WE', 'LQ', 'UU']), 'HU': (0, ['PW', 'GE']), 'PC': (0, ['AA', 'YI']), 'FT': (0, ['XI', 'VG']), 'AY': (19, ['ZX', 'CY', 'FG', 'DO', 'YO']), 'RT': (21, ['YA', 'BO']), 'ZK': (0, ['GC',

In [ ]:
# part 2
import itertools
import copy

def act(dP, clock, loc, openset):
    options_A = (*valves[loc[0]][1], 'open')
    options_B = (*valves[loc[1]][1], 'open')
    for r in itertools.product(options_A, options_B):
        newloc_A, newloc_B = loc #, newloc_B = loc
        # newloc_B = loc[1]
        dPnew = dP
        go_flag = True  # assume we can push todo unless one combo is open and can't open
        # if r[0] != r[1]:
        #     continue

        if r[0] != 'open' and r[1] != 'open' and (dP, clock, (r[0], r[1]), openset) not in todo:
            newloc_A, newloc_B = r
            newset = openset

        elif r[0] == 'open' and r[1] == 'open' and loc[0] not in openset and valves[loc[0]][0] > 0 and loc[1] not in openset and valves[loc[1]][0] > 0:
            dPnew -= valves[loc[0]][0]*(TOTAL_TIME-clock)
            newset = (*openset,loc[0])
            if loc[1] != loc[0]:
                dPnew -= valves[loc[1]][0]*(TOTAL_TIME-clock)
                newset = (*newset,loc[1])

        elif r[0] == 'open' and r[1] != 'open' and loc[0] not in openset and valves[loc[0]][0] > 0:
            dPnew -= valves[loc[0]][0]*(TOTAL_TIME-clock)
            newset = (*openset,loc[0])
            newloc_B = r[1]

        elif r[0] != 'open' and r[1] == 'open' and loc[1] not in openset and valves[loc[1]][0] > 0:
            dPnew -= valves[loc[1]][0]*(TOTAL_TIME-clock)
            newset = (*openset,loc[1])
            newloc_A = r[0]

        else:
            go_flag = False

        if go_flag and newloc_A != newloc_B:
            if (  ((newloc_A,newloc_B),newset) not in visited
                or ( ((newloc_A,newloc_B),newset) in visited and clock < visited[((newloc_A,newloc_B),newset)] )  ):

                if (  ((newloc_B,newloc_A),newset) not in visited
                    or ( ((newloc_B,newloc_A),newset) in visited and clock < visited[((newloc_B,newloc_A),newset)] )  ):
                    heappush(todo,(dPnew, clock, (newloc_A, newloc_B), newset))

TOTAL_TIME = 26  # part 1: 30, # part 2: 26
print(valves)
# completed = [] # heap of completed states
dP_biggest_drop = 0
visited = {}  # dictionary of cache of visited, key: loc,openset, value: clock
todo = [(0,0,('AA','AA'),())]  # starting state: (dP, clock, loc, openset)
clock = 0
cnt = 0
while todo != []:
    cnt += 1
    dP, clock, loc, openset = heappop(todo)
    if cnt % 1e7 == 0:
        print(f'cnt = {cnt}, current pop = {dP, clock, loc, openset}')
    # if loc[0] != loc[1]:  # part 1, but set TOTAL_TIME = 30
    #     continue
    # if loc[0] == loc[1] and clock > 0:
    #     continue
    # if (loc,openset) in visited and clock > visited[(loc,openset)]:
    #     continue # move on to next pop
    # if ((loc[::-1]),openset) in visited and clock > visited[((loc[::-1]),openset)]:
    #     continue # move on to next pop
    if dP < dP_biggest_drop:
        dP_biggest_drop = dP
        print(f'Best so far: {dP, clock, loc, openset}')
    visited[(loc,openset)] = clock
    clock += 1
    if clock <= TOTAL_TIME-1:
        act(dP, clock, loc, openset)

print(f'cnt = {cnt}, current pop = {dP, clock, loc, openset}')

# should get for part 1:
# Best so far: (-2114, 28, 'PW', ('AY', 'GC', 'WT', 'SY', 'VR', 'CB', 'PW'))
# cnt = 10000000, current pop = (-896, 28, 'HS', ('GC', 'XI', 'SY', 'WT'))
# cnt = 12171829, current pop = (0, 14, 'YA', ())

# part 2 example: 1.6 sec
# Best so far: (-1707, 11, ('EE', 'AA'), ('DD', 'JJ', 'HH', 'BB', 'CC', 'EE'))
# cnt = 274949, current pop = (0, 6, ('GG', 'GG'), ())

# part 2: 32 minutes, cnt=50M, kernel crashed
# TOO LOW: Best so far: (-2213, 22, ('HS', 'RT'), ('AY', 'YI', 'CB', 'GC', 'VR', 'QT', 'WT', 'VG', 'QO', 'SY', 'RT'))


{'SY': (23, ['CS']), 'VG': (15, ['XZ', 'FT', 'PF', 'TF', 'LG']), 'LQ': (0, ['YI', 'PW']), 'ON': (0, ['AA', 'KC']), 'EX': (0, ['GC', 'CB']), 'JV': (0, ['XI', 'QT']), 'UU': (0, ['YI', 'CB']), 'JS': (8, ['WE', 'JN', 'YN', 'KC', 'WC']), 'WE': (0, ['YI', 'JS']), 'YN': (0, ['JS', 'PW']), 'LG': (0, ['BO', 'VG']), 'GE': (0, ['AA', 'HU']), 'AA': (0, ['VP', 'ON', 'PC', 'GE', 'YO']), 'GC': (20, ['DG', 'ZK', 'WC', 'KF', 'EX']), 'QO': (12, ['YA', 'TF', 'BV']), 'JN': (0, ['JS', 'ZX']), 'WO': (0, ['ZW', 'XI']), 'YO': (0, ['AA', 'AY']), 'TF': (0, ['VG', 'QO']), 'PF': (0, ['BY', 'VG']), 'DO': (0, ['KF', 'AY']), 'FG': (0, ['AY', 'PW']), 'VR': (24, ['IC', 'CH']), 'CH': (0, ['VR', 'NW']), 'CB': (9, ['QZ', 'BP', 'UU', 'JW', 'EX']), 'YA': (0, ['QO', 'RT']), 'KF': (0, ['DO', 'GC']), 'DP': (0, ['QT', 'ZK']), 'YI': (3, ['PC', 'CY', 'WE', 'LQ', 'UU']), 'HU': (0, ['PW', 'GE']), 'PC': (0, ['AA', 'YI']), 'FT': (0, ['XI', 'VG']), 'AY': (19, ['ZX', 'CY', 'FG', 'DO', 'YO']), 'RT': (21, ['YA', 'BO']), 'ZK': (0, ['GC',

: 